<a href="https://colab.research.google.com/github/cobang0111/2023ESWContest_humanoid_2014/blob/main/4_stable_baseline3_SAC_sgminicat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade pybullet==3.1.6 imageio-ffmpeg stable-baselines3

import os
import time
import math

import numpy as np
import imageio_ffmpeg
from PIL import Image
import matplotlib.pyplot as plt
import cv2
from base64 import b64encode
from IPython.display import HTML
import gym
from gym import spaces

import pybullet
import pybullet_data
#from stable_baselines3.sac.policies import MlpPolicy
#from stable_baselines3 import SAC

In [2]:
!pip install 'shimmy>=0.2.1'

In [46]:
from stable_baselines3.sac.policies import MlpPolicy
from stable_baselines3 import SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.results_plotter import load_results, ts2xy
log_dir = "/tmp/gym/"
os.makedirs(log_dir, exist_ok=True)

# Simulation & Environment
FLAG = pybullet.URDF_USE_SELF_COLLISION_EXCLUDE_PARENT | pybullet.URDF_USE_SELF_COLLISION


# camera parameters
cam_target_pos = [0, -0.2, 0.2]
cam_distance = 2.5
cam_yaw, cam_pitch, cam_roll = 60, -20, 0
cam_width, cam_height = 480, 360
cam_up, cam_up_axis_idx, cam_near_plane, cam_far_plane, cam_fov = [0, 0, 0], 2, 0.01, 100, 60


pybullet.connect(pybullet.DIRECT) #Open directly without GUI
pybullet.resetSimulation()
pybullet.setAdditionalSearchPath(pybullet_data.getDataPath())
pybullet.setTimeStep(1/400)
pybullet.setGravity(0, 0, -9.81)

plane = pybullet.loadURDF("plane.urdf")
robot = pybullet.loadURDF("/content/112101_3/urdf/112101_3.urdf",[0,0,0.9], useFixedBase=0, flags = FLAG)


class minicatEnv(gym.Env):

    def __init__(self, initial_position):
        super(minicatEnv, self).__init__()
        self.action_space = spaces.Box(low=-100, high=100, shape=(2, ))
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(3, 2))
        self.previous_position = initial_position

    def position_update(self, new_position):
        difference = new_position - self.previous_position
        self.previous_position = new_position
        return difference

    def reset(self):

        self.steps = 0

        # Initializing
        pybullet.setJointMotorControl2(robot, 0, pybullet.VELOCITY_CONTROL, force = 0)
        pybullet.setJointMotorControl2(robot, 1, pybullet.VELOCITY_CONTROL, force = 0)

        pybullet.changeVisualShape(robot, 0, rgbaColor=[0.65, 0.5, 0.8, 1])
        pybullet.changeVisualShape(robot, 1, rgbaColor=[0.45, 0.3, 0.6, 1])

        # friction
        pybullet.changeDynamics(plane, -1, lateralFriction=0.5)
        #for joint_index in range(pybullet.getNumJoints(robot)):
            #pybullet.setJointMotorControl2(self.robot, joint_index, pybullet.POSITION_CONTROL, force = 0)
            #pybullet.changeDynamics(self.robot, joint_index, lateralFriction=0.2376545)
        pybullet.changeDynamics(robot, 2, lateralFriction=0.5)

        initial_angle_0 = np.radians(-30)
        initial_angle_1 = np.radians(60)

        pybullet.resetJointState(robot, 0, initial_angle_0) # Set the initial angle of the joint
        pybullet.resetJointState(robot, 1, initial_angle_1)
        pybullet.resetBasePositionAndOrientation(robot, [0, 0, 0.9], [0, 0, 0, 1])

        joint_info_0 = pybullet.getJointState(robot, 0)  # Get the initial joint state
        joint_info_1 = pybullet.getJointState(robot, 1)

        joint_angle = [pybullet.getJointState(robot, i)[0] for i in range(2)]
        joint_velocity = [pybullet.getJointState(robot, i)[1] for i in range(2)]
        joint_reaction = [pybullet.getJointState(robot, i)[2] for i in range(2)]
        joint_torque = [pybullet.getJointState(robot, i)[3] for i in range(2)]

        arr = [joint_angle, joint_velocity, joint_torque]

        return np.array(arr)

    def step(self, action):

        torque2 = action[0]
        torque3 = action[1]
        pybullet.setJointMotorControl2(robot, 0, pybullet.TORQUE_CONTROL, force = torque2)
        pybullet.setJointMotorControl2(robot, 1, pybullet.TORQUE_CONTROL, force = torque3)

        # Step the simulation
        pybullet.stepSimulation()

        # Get the joint's angle and velocity
        joint_info_0 = pybullet.getJointState(robot, 0)
        joint_info_1 = pybullet.getJointState(robot, 1)

        joint_angle = [pybullet.getJointState(robot, i)[0] for i in range(2)]
        joint_velocity = [pybullet.getJointState(robot, i)[1] for i in range(2)]
        joint_reaction = [pybullet.getJointState(robot, i)[2] for i in range(2)]
        joint_torque = [pybullet.getJointState(robot, i)[3] for i in range(2)]
        joint_x = pybullet.getBasePositionAndOrientation(robot)[0][0]
        joint_y = pybullet.getBasePositionAndOrientation(robot)[0][1]
        joint_z = pybullet.getBasePositionAndOrientation(robot)[0][2]

        pos_diff_x = self.position_update(joint_x)
        #pos_diff_z = self.position_update(joint_z)

        self.steps += 1

        # Calculate reward
        healthy_reward = 1*0.1*self.steps
        forward_reward = 1 * pos_diff_x/(1/400)

        reward = healthy_reward + forward_reward

        # The episode is done if the pole is more than some degrees from vertical
        done = 0

        if abs(joint_angle[0]) > np.radians(70):
            done = 1

        if self.steps > 100000:
            done = 1

        if joint_z < 0.5:
            done = 1

        if joint_z > 2:
            done = 1

        if abs(joint_x) > 5:
            done = 1

        if abs(joint_y) > 0.5:
            done = 1

        if done:
            print(round(reward, 1))
            arr = self.reset()
            #print(arr)

        else : arr = [joint_angle, joint_velocity, joint_torque]

        return np.array(arr), reward, done, {}


    def render(self):
        cam_view_matrix = pybullet.computeViewMatrixFromYawPitchRoll(cam_target_pos, cam_distance, cam_yaw, cam_pitch, cam_roll, cam_up_axis_idx)
        cam_projection_matrix = pybullet.computeProjectionMatrixFOV(cam_fov, cam_width*1./cam_height, cam_near_plane, cam_far_plane)
        image = pybullet.getCameraImage(cam_width, cam_height, cam_view_matrix, cam_projection_matrix)[2][:, :, :3]
        image = np.ascontiguousarray(image)
        return image

    def close(self):
        pybullet.disconnect()

# Create the environment
env = minicatEnv(initial_position = 0)

model = SAC('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=100000)
#plot_results(log_dir+'monitor.csv')
model.save("sac_minicat_env")

del model # remove to demonstrate saving and loading

model = SAC.load("sac_minicat_env")

vid = imageio_ffmpeg.write_frames('vid.mp4', (cam_width, cam_height), fps=30)
vid.send(None) # seed the video writer with a blank frame


# Enjoy trained agent
for i in range(1):
    obs = env.reset()
    for j in range(2400):
        action, _states = model.predict(obs)
        #print(action)
        obs, rewards, done, info = env.step(action)
        if j % 8 == 0: # pybullet steps at 240fps, but we want 30fps for video
            image = env.render()
            vid.send(image)
        if done:
            obs = env.reset()

vid.close()
env.close()


Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
3.0
0.8
3.0
2.2
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 23.2     |
|    ep_rew_mean     | 31.9     |
| time/              |          |
|    episodes        | 4        |
|    fps             | 2214     |
|    time_elapsed    | 0        |
|    total_timesteps | 93       |
---------------------------------
0.8
2.4
1.9
1.7
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 20.1     |
|    ep_rew_mean     | 24.4     |
| time/              |          |
|    episodes        | 8        |
|    fps             | 192      |
|    time_elapsed    | 0        |
|    total_timesteps | 161      |
| train/             |          |
|    actor_loss      | -3.33    |
|    critic_loss     | 1.94     |
|    ent_coef        | 0.983    |
|    ent_coef_loss   | -0.0565  |
|    learning_rate   | 0.0003   |
|    n_updates       | 

11.5
12.4
12.1
11.5
11.3
12.0
11.0
11.0
12.4
12.8
11.7
12.2
11.0
11.3
11.5
11.1
11.7
11.9
11.8
11.1


In [47]:
# Play recorded video

mp4 = open('vid.mp4', 'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML('<video width=480 controls><source src="%s" type="video/mp4"></video>' % data_url)